In [1]:
FACE_DATA_PATH=r'G:\forgithub\SiameseNet\datasets\Extracted Faces\Extracted Faces'
EXTRACTED_FACES_PATH =r'G:\forgithub\SiameseNet\datasets\Face Data\Face Dataset'

In [2]:
import os
import zipfile
import shutil

import random
import math
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.losses import Loss
from tensorflow.keras.applications import EfficientNetV2B3
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import numpy as np

In [3]:
def explore_folder(folder_path):
    print(f'Exploring {os.path.basename(folder_path)}')
    image_shapes = []
    num_images = 0
    num_people = 0
    for folder_name in os.listdir(folder_path):
        subfolder_path = os.path.join(folder_path, folder_name)
        for image_name in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, image_name)
            image = cv2.imread(image_path)
            image_shapes.append(image.shape)
            num_images += 1
        num_people +=1

    print(f'Unique image shapes in: {set(image_shapes)}')
    print(f"Total number of images: {num_images}")
    print(f"Total number of people: {num_people}")
    return image_shapes, num_images, num_people

In [4]:
explore_folder(FACE_DATA_PATH);

Exploring Extracted Faces
Unique image shapes in: {(128, 128, 3)}
Total number of images: 6107
Total number of people: 1324


In [5]:
DATASET = 'images/output_dataset'
if os.path.exists(DATASET):
    shutil.rmtree(DATASET)
os.makedirs(DATASET)

def copy_to_output_dataset(input_path, output_path):
    for person_folder in os.listdir(input_path):
        person_folder_path = os.path.join(input_path, person_folder)
        if os.path.isdir(person_folder_path):
            output_person_folder = os.path.join(output_path, person_folder)
            if not os.path.exists(output_person_folder):
                os.makedirs(output_person_folder)

            for image_file in os.listdir(person_folder_path):
                if image_file.endswith('.jpg'):
                    src_image_path = os.path.join(person_folder_path, image_file)
                    dst_image_path = os.path.join(output_person_folder, image_file)
                    if os.path.exists(dst_image_path):
                        base, ext = os.path.splitext(dst_image_path)
                        dst_image_path = f"{base}_1{ext}"
                    shutil.copy(src_image_path, dst_image_path)

copy_to_output_dataset(FACE_DATA_PATH, DATASET)
copy_to_output_dataset(EXTRACTED_FACES_PATH, DATASET)

In [6]:
def triplets(folder_paths, max_triplets=7):
    anchor_images = []
    positive_images = []
    negative_images = []

    for person_folder in folder_paths:
        images = [os.path.join(person_folder, img)
                  for img in os.listdir(person_folder)]
        num_images = len(images)

        if num_images < 2:
            continue

        random.shuffle(images)

        for _ in range(max(num_images-1, max_triplets)):
            anchor_image = random.choice(images)

            positive_image = random.choice([x for x in images
                                            if x != anchor_image])

            negative_folder = random.choice([x for x in folder_paths
                                             if x != person_folder])

            negative_image = random.choice([os.path.join(negative_folder, img)
                                            for img in os.listdir(negative_folder)])

            anchor_images.append(anchor_image)
            positive_images.append(positive_image)
            negative_images.append(negative_image)

    return anchor_images, positive_images, negative_images

In [7]:
person_folders = [os.path.join(DATASET, folder_name)
                  for folder_name in os.listdir(DATASET)]

anchors, positives, negatives = triplets(person_folders)

In [8]:
len(anchors), len(positives), len(negatives)

(17318, 17318, 17318)

In [9]:
def split_triplets(anchors,
                   positives,
                   negatives,
                   validation_split=0.2):

    triplets = list(zip(anchors, positives, negatives))

    train_triplets, val_triplets = train_test_split(triplets,
                                                    test_size=validation_split,
                                                    random_state=42)

    return train_triplets, val_triplets

In [10]:
train_triplets, val_triplets = split_triplets(anchors,
                                              positives,
                                              negatives)
len(train_triplets), len(val_triplets)

(13854, 3464)

In [11]:
def load_and_preprocess_image(image_path, expand_dims=False):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image not found or could not be loaded: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128, 128))
    if expand_dims:
        image = np.expand_dims(image, axis=0)
    return image


def batch_generator(triplets, batch_size=32, augment=True):
    total_triplets = len(triplets)
    random_indices = list(range(total_triplets))
    random.shuffle(random_indices)

    datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.05,
        height_shift_range=0.05,
        horizontal_flip=True,
        zoom_range=0.2
    )

    for i in range(0, total_triplets, batch_size):
        batch_indices = random_indices[i:i + batch_size]
        if len(batch_indices) < batch_size:
            continue  # Skip the last incomplete batch

        batch_triplets = [triplets[j] for j in batch_indices]
        anchor_batch, positive_batch, negative_batch = [], [], []

        for triplet in batch_triplets:
            anchor, positive, negative = triplet

            anchor_image = load_and_preprocess_image(anchor)
            positive_image = load_and_preprocess_image(positive)
            negative_image = load_and_preprocess_image(negative)

            if augment:
                anchor_image = datagen.random_transform(anchor_image)
                positive_image = datagen.random_transform(positive_image)
                negative_image = datagen.random_transform(negative_image)

            anchor_batch.append(anchor_image)
            positive_batch.append(positive_image)
            negative_batch.append(negative_image)

        yield (np.array(anchor_batch), np.array(positive_batch), np.array(negative_batch)), np.ones((len(batch_indices), 1))

In [13]:
# Check if a GPU is available
gpus = tf.config.experimental.list_physical_devices('GPU')

if gpus:
    print("GPU is available.")
    # Optionally set memory growth to avoid OOM errors
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPU.")
    except RuntimeError as e:
        print(e)
else:
    print("GPU is not available, using CPU.")

GPU is available.
Memory growth set for GPU.


In [35]:
# Embedding Model
class EmbeddingModel(Model):
    def __init__(self, embedding_size=128, dropout_rate=0.5):
        super(EmbeddingModel, self).__init__()

        # Use EfficientNetV2B3 as the backbone for feature extraction
        self.feature_extractor = EfficientNetV2B3(input_shape=(128, 128, 3),
                                                  include_top=False,
                                                  weights='imagenet')
        
        # Freeze the layers of EfficientNetV2B3
        self.feature_extractor.trainable = False

        # Add flattening layer for converting 2D feature maps to 1D vectors
        self.flatten = Flatten(name="flatten_layer")

        self.fc1 = Dense(512, activation='relu', name="fc1_dense")
        self.batchnorm1 = BatchNormalization(name="batchnorm1")
        self.dropout1 = Dropout(dropout_rate, name="dropout1")

        self.fc2 = Dense(256, activation='relu', name="fc2_dense")
        self.batchnorm2 = BatchNormalization(name="batchnorm2")
        self.dropout2 = Dropout(dropout_rate, name="dropout2")

        # Output embedding layer with custom embedding size
        self.embedding_output = Dense(embedding_size, name="embedding_output")

    def call(self, inputs):
        # Pass inputs through feature extractor (EfficientNetV2B3)
        x = self.feature_extractor(inputs)
        
        # Flatten the output feature map
        x = self.flatten(x)
        
        # Fully connected layers with batch normalization and dropout
        x = self.fc1(x)
        x = self.batchnorm1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.batchnorm2(x)
        x = self.dropout2(x)

        # Return the final embedding
        return self.embedding_output(x)

# Custom Triplet Loss Layer
class TripletLayer(Layer):
    def __init__(self, margin=1.0, **kwargs):
        super(TripletLayer, self).__init__(**kwargs)
        self.margin = margin

    def call(self, anchor_embeddings, positive_embeddings, negative_embeddings):
        # Calculate squared Euclidean distance between anchor and positive/negative samples
        positive_distance = tf.reduce_sum(tf.square(anchor_embeddings - positive_embeddings), axis=1)
        negative_distance = tf.reduce_sum(tf.square(anchor_embeddings - negative_embeddings), axis=1)

        # Triplet loss: max(positive_distance - negative_distance + margin, 0)
        triplet_loss = tf.maximum(positive_distance - negative_distance + self.margin, 0.0)

        return tf.reduce_mean(triplet_loss)

# Triplet Siamese Model
class TripletSiameseModel(Model):
    def __init__(self, embedding_size=128, margin=1.0, dropout_rate=0.5):
        super(TripletSiameseModel, self).__init__()
        
        # Base model for generating embeddings
        self.embedding_model = EmbeddingModel(embedding_size=embedding_size, dropout_rate=dropout_rate)
        
        # Triplet loss layer with the specified margin
        self.triplet_loss_layer = TripletLayer(margin=margin, name="triplet_loss_layer")

    def call(self, inputs):
        # Unpack the inputs into anchor, positive, and negative examples
        anchor_input, positive_input, negative_input = inputs

        # Generate embeddings for anchor, positive, and negative examples
        anchor_embeddings = self.embedding_model(anchor_input)
        positive_embeddings = self.embedding_model(positive_input)
        negative_embeddings = self.embedding_model(negative_input)

        # Return the triplet loss
        return self.triplet_loss_layer(anchor_embeddings, positive_embeddings, negative_embeddings)

    # Build the model graph for easy visualization
    def build_graph(self):
        anchor_input = Input(shape=(128, 128, 3), name="anchor_input")
        positive_input = Input(shape=(128, 128, 3), name="positive_input")
        negative_input = Input(shape=(128, 128, 3), name="negative_input")
        
        # Create the model
        return Model(inputs=[anchor_input, positive_input, negative_input], 
                     outputs=self.call([anchor_input, positive_input, negative_input]))

# Identity Loss
def identity_loss(y_true, y_pred):
    return tf.reduce_mean(y_pred)


In [36]:
input_test = np.random.rand(1, 128, 128, 3)
model = baseMobileModel()
model(input_test).shape

TensorShape([1, 128])

In [37]:
triplet_layer = TripletLayer(margin=1)
anchors = tf.random.normal((5, 128))
positives = tf.random.normal((5, 128))
negatives = tf.random.normal((5, 128))
loss = triplet_layer(anchors, positives, negatives)

print("Triplet Loss:", loss.numpy())

Triplet Loss: 7.421262


In [38]:
loss_tracker = identity_loss
optimizer = Adam(0.001)

In [39]:
siamese_net = TripletSiameseModel(embedding_size=256, margin=0.5)


  0%|          | 0/109 [05:05<?, ?it/s]


In [42]:
class Trainer:
    def __init__(self, batch_generator, model, loss_fun, optimizer):
        """
        Initializes the Trainer class with a model, loss function, and optimizer.
        Args:
            model: Siamese model.
            loss_fun: The loss function to optimize.
            optimizer: The optimizer to apply during training.
        """
        self.model = model
        self.loss_fun = loss_fun
        self.optimizer = optimizer
        self.batch_generator = batch_generator

    def __call__(self, train_triplets, val_triplets, epochs, batch_size):
        """
        Executes the training and validation loops for the specified epochs and batch size.
        Args:
            train_triplets: Training data in triplet format (anchor, positive, negative).
            val_triplets: Validation data in triplet format (anchor, positive, negative).
            epochs: Number of epochs to train the model.
            batch_size: Size of each batch during training and validation.
        Returns:
            A tuple containing lists of training and validation losses for each epoch.
        """
        train_epochs_losses = []
        val_epochs_losses = []

        # Compute steps per epoch
        train_steps_per_epoch = math.ceil(len(train_triplets) / batch_size)
        val_steps_per_epoch = math.ceil(len(val_triplets) / batch_size)

        for epoch in range(epochs):
            # Initialize generators for training and validation
            train_generator = self.batch_generator(train_triplets, batch_size=batch_size, augment=True)
            val_generator = self.batch_generator(val_triplets, batch_size=batch_size, augment=False)
            print(f"\nEpoch {epoch + 1}/{epochs}")

            # Training step
            train_losses = self.train_step_for_one_epoch(train_generator, train_steps_per_epoch)
            avg_train_loss = np.mean(train_losses)
            train_epochs_losses.append(avg_train_loss)
            print(f"Average Training Loss: {avg_train_loss:.4f}")

            # Validation step
            val_losses = self.perform_validation(val_generator, val_steps_per_epoch)
            avg_val_loss = np.mean(val_losses)
            val_epochs_losses.append(avg_val_loss)
            print(f"Average Validation Loss: {avg_val_loss:.4f}")

        return train_epochs_losses, val_epochs_losses

    @tf.function
    def apply_gradients(self, anchors, positives, negatives, labels):
        """
        Applies gradients and updates the model weights using the optimizer.
        Args:
            anchors: Anchor images (input).
            positives: Positive images (input).
            negatives: Negative images (input).
            labels: Labels for the triplet loss.
        Returns:
            The predicted values and the computed loss for the batch.
        """
        with tf.GradientTape() as tape:
            y_pred = self.model([anchors, positives, negatives], training=True)
            loss = self.loss_fun(labels, y_pred)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return y_pred, loss

    def train_step_for_one_epoch(self, train_generator, train_steps_per_epoch):
        """
        Performs one epoch of training across all batches.
        Args:
            train_generator: Generator that yields batches of training data.
            train_steps_per_epoch: Number of training steps in one epoch.
        Returns:
            A list of losses for each batch in the epoch.
        """
        losses = []
        pbar = tqdm(total=train_steps_per_epoch, position=0, leave=True)
        for step, ((anchors, positives, negatives), labels) in enumerate(train_generator):
            # Convert to TensorFlow tensors if not already
            anchors, positives, negatives, labels = map(tf.convert_to_tensor, (anchors, positives, negatives, labels))
            
            y_pred, loss = self.apply_gradients(anchors, positives, negatives, labels)
            losses.append(loss)  # Convert Tensor loss to a numpy float

            pbar.set_description(f"Training Loss: {float(loss):.4f}")
            pbar.update(1)
        pbar.close()
        return losses
        
    
    def perform_validation(self, val_generator, val_steps_per_epoch):
        """
        Performs validation across all batches.
        Args:
            val_generator: Generator that yields batches of validation data.
            val_steps_per_epoch: Number of validation steps in one epoch.
        Returns:
            A list of losses for each batch in the validation epoch.
        """
        losses = []
        pbar = tqdm(total=val_steps_per_epoch, position=0, leave=True)
        for step, ((anchors, positives, negatives), labels) in enumerate(val_generator):
            # Convert to TensorFlow tensors if not already
            anchors, positives, negatives, labels = map(tf.convert_to_tensor, (anchors, positives, negatives, labels))

            y_pred = self.model([anchors, positives, negatives], training=False)
            loss = self.loss_fun(labels, y_pred)
            
            # Convert the loss tensor to a numpy float for logging and storing
            loss_value = loss.numpy()  # Ensure the tensor is converted to a NumPy value
            losses.append(loss_value)

            pbar.set_description(f"Validation Loss: {loss_value:.4f}")
            pbar.update(1)
        pbar.close()
        return losses

In [ ]:
trainer = Trainer(
    batch_generator,
    siamese_net,
    identity_loss,
    optimizer
  )

his_losses = trainer(
    train_triplets,
    val_triplets,
    epochs=30,
    batch_size=64
  )


Epoch 1/30


Training Loss: 1.3496: 100%|█████████▉| 216/217 [04:41<00:01,  1.30s/it]


Average Training Loss: 2.2107


Validation Loss: 0.6112:  98%|█████████▊| 54/55 [00:46<00:00,  1.17it/s]


Average Validation Loss: 1.0824

Epoch 2/30


...

Training Loss: 0.2320: 100%|█████████▉| 216/217 [04:18<00:01,  1.20s/it]


Average Training Loss: 0.2451


Validation Loss: 0.1281:  98%|█████████▊| 54/55 [00:54<00:01,  1.02s/it]


Average Validation Loss: 0.1469

Epoch 26/30


Training Loss: 0.3321:  15%|█▌        | 33/217 [00:40<03:37,  1.18s/it]

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to read, resize, normalize, and reshape an image
def process_image(image_path):
    # Read the image
    image = cv2.imread(image_path)
    
    # Resize the image to 128x128
    image_resized = cv2.resize(image, (128, 128))
    
    # Normalize the image to [0, 1]
    image_normalized = image_resized / 255.0
    
    # Reshape the image to (1, 128, 128, 3)
    image_reshaped = image_normalized[np.newaxis, ...]  # Add new axis

    return image_reshaped

# List of image paths
image_paths = ['/content/hos1.PNG', '/content/hos2.PNG', '/content/hos3.PNG', '/content/allam.PNG', '/content/donia.PNG']

# Process the images
processed_images = [process_image(path) for path in image_paths]

# Show the processed images
plt.figure(figsize=(10, 10))

for i, img in enumerate(processed_images):
    plt.subplot(1, len(processed_images), i + 1)
    plt.imshow(img[0])  # Remove the first dimension for display
    plt.axis('off')     # Hide axis

plt.show()
